In [1]:
pwd

'/Users/herculesgao/Desktop/Information_Tracer'

In [13]:
import requests
from datetime import date, timedelta
import time
from pprint import pprint
import json
import os
import pandas as pd 
import bs4 as bs
import requests
import yfinance as yf
import datetime
from datetime import datetime as dt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

SUBMIT_URL = 'https://informationtracer.com/submit'
STATUS_URL = 'https://informationtracer.com/status'
RESULT_URL = 'https://informationtracer.com/result'


def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score
# STEP 1: submit a search query, and get id_hash256 (a unique identifier)
def step_1_submit(query, token, start_date, end_date):
    id_hash256 = None
    try:
        response = requests.post(SUBMIT_URL, 
                             timeout=10,
                             json={'query': query, 
                                   'token': token,
                                   'start_date': start_date,
                                    'end_date': end_date,
                                    'twitter_only': "true"
                                   }                                   
                            )
        if 'id_hash256' in response.json():
            id_hash256 = response.json()['id_hash256']
        else:
            print("Submission failed!")
            print(response.json())

    except Exception as e:
        print("Submission failed!")
        print(e)
    
    return id_hash256
        

# STEP 2: periodically check status, and get partial results (first 5 tweets)
def step_2_check_status(id_hash256, token):
    task_status = None
    MAX_ROUND = 40
    current_round = 0
    include_partial_results = 1

    while task_status != 'finished' and  current_round < MAX_ROUND:            
        current_round += 1
#         print(current_round)
        try:
            full_url = '{}?id_hash256={}&token={}&include_partial_results={}'.format(
                STATUS_URL, id_hash256, token, include_partial_results
                )
            response = requests.get(full_url, timeout=10).json()            
#             print(response)
            if 'status' in response:
                # NOTE: can render status_percentage, status_text on the UI
                task_status = response['status']
                status = response['status']
                status_percentage = response['status_percentage']
                status_text = response['status_text']
                tweet_preview = response.get('tweet_preview', None)
#                 if tweet_preview:
                    # NOTE: can render tweet_preview on the UI            
#                     print('received {} partial tweets'.format(len(tweet_preview)))
                    # TO save bandwidth, we can tell the API not to return partial results
#                     include_partial_results = 0

                if status != 'finished':
                    time.sleep(6)
                else:
                    return 'finished'

            else:
                print('status is not in response')
                pprint(response)
                return 'failed'

        except Exception as e:
            print('Exception when checking job status!')
            print(e)

    return 'timeout'


# STEP 3: get full results
def step_3_get_result(id_hash256, token):
    try:
        response = requests.get('{}?token={}&id_hash256={}'.format(RESULT_URL, token, id_hash256), timeout=10)
#         pprint(response.json().keys())
#         pprint(response.json())
        return response.json()
    except Exception as e:
        print(e)

    return []

def step_b1_loadTicker():
    try: 
        resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
        soup = bs.BeautifulSoup(resp.text, 'lxml')
        table = soup.find('table', {'class': 'wikitable sortable'})
        table.findAll('tr')[1:][0].findAll('td')[1].text
    except: 
        print("cannot get the table from Wiki!")
    tickers = []
    companies = []
    impcs = []
    impps = []
    dc = {}
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        company = row.findAll('td')[1].text
        tickers.append(ticker)
        companies.append(company)
        dc[ticker.replace('\n', '').replace('.', '-')] = company
    tickers = [s.replace('\n', '').replace('.', '-') for s in tickers]
    start =  dt.now()
    end = dt.now()+timedelta(days = 1)
    data = yf.download(tickers, start=start, end=end)
    

    df_stock = data.stack().reset_index().rename(index=str, columns={"level_1": "Symbol"}).sort_values(['Symbol','Date'])
    n = 0 
    for ticker in df_stock['Symbol']: 
        n +=1 
        last_p = df_stock[df_stock['Symbol']==ticker]['Adj Close'].iloc[-1] #  last price
        print(n, "Ticker",ticker)
        ticker = yf.Ticker(ticker)
        if ticker.options:  # checking if there is no options
            opt = ticker.option_chain(ticker.options[0])
            calls = opt.calls
            puts = opt.puts
            try: 
                impc = calls[calls['strike'] <= last_p].iloc[-1]['impliedVolatility']
            except Exception as e: 
                print("Calls",e)
                impc = 0 
            try: 
                impp = puts[puts['strike'] >= last_p].iloc[0]['impliedVolatility']
            except Exception as e: 
                print("Puts",e)
                impp = 0 
        else: 
            impc = 0 
            impp = 0 
        impcs.append(impc)
        impps.append(impp)
    df_stock['ImpliedVolcalls'] = impcs
    df_stock['ImpliedVolputs'] = impps
    #     df_stock.set_index('Date', inplace=True)
    listCompany = [dc[c] for c in df_stock['Symbol']]
    df_stock['company'] = listCompany
    return df_stock.reset_index(drop = True)


In [19]:
if __name__ == '__main__':
    while True:
        if dt.now().hour == 16: 
            program_start_time = dt.now()
            print("Pulling all the SPY stock")
            df_stock = step_b1_loadTicker()
            print("Doen creating SPY 500 stock!")
        #     df_stock['Sentiment_chatgpt'] = ''
            df_stock['Sentiment_Vader'] = ''
            df_stock['Posts_t'] = ''
            df_stock['interaction'] = ''

            queries = [ "$" + i + " AND " + "("+u+")" if len(u.split(" "))==1 else "$" + i + " AND " + "("+" ".join(u.split(" ")[:-1])+")" for i,u in zip(df_stock['Symbol'], df_stock['company'])]
            token = 'r3myxh31yzy2py5k6qg3u0k59xbkmmpu'
            print('begin getting social media feed')
            for i in range(len(df_stock['Symbol'])):
                print(i,df_stock['Symbol'][i])
                start_date = str(df_stock['Date'][i].date()-timedelta(days =1))
                end_date = str(df_stock['Date'][i].date())
                id_hash256 = step_1_submit(queries[i], token, start_date, end_date)
                if id_hash256:
                    print(id_hash256)
                    status = step_2_check_status(id_hash256, token)
                    print(status)
                    if status == 'finished':
                        print("get social media feeds for",token)
                        result = step_3_get_result(id_hash256, token)
                        minS, maxS = 0,0
                        for j in range(len(result['posts']['twitter'])):
                            compound = sentiment_analyzer_scores(result['posts']['twitter'][j]['d'])['compound']
                            if compound< minS: 
                                minS = compound
                            elif compound > maxS:
                                maxS = compound
                        Vader_score = maxS+ minS

        #                 df_stock.loc[i,'Sentiment_chatgpt'] =result['sentiment']['twitter']
                        df_stock.loc[i, 'Posts_t'] = str(result['posts']['twitter'])
                        df_stock.loc[i, 'Sentiment_Vader'] =Vader_score
                        df_stock.loc[i, 'interaction'] = result['interaction']['twitter']
            program_end_time = dt.now()
            print('Duration: {}'.format(program_end_time - program_start_time))
            print("Saving the file!")
            new_string = str(dt.now().date()).replace("-","_")
            file_name = 'SPY500_stock_' +new_string+'.csv'
            df_stock.to_csv(file_name)  
                    # NOTE: do something with result
            break 
        else:
            time.sleep(5)

KeyboardInterrupt: 

In [18]:
df_stock

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,ImpliedVolcalls,ImpliedVolputs,company,Sentiment_Vader,Posts_t,interaction,Changes
0,2024-01-11,A,129.679993,129.679993,130.679993,127.900002,130.580002,2059649,0.478033,0.262703,Agilent Technologies,0.0000,[],0,-1.410004
1,2024-01-11,AAL,14.590000,14.590000,14.630000,14.240100,14.480000,32679869,0.468755,0.523442,American Airlines Group,-0.2481,"[{'d': 'And 4 more, for the road, via:\n$AAL #...",4,0.240000
2,2024-01-11,AAPL,185.589996,185.589996,187.050003,183.619995,186.539993,48302261,0.159676,0.103525,Apple Inc.,0.1550,"[{'d': 'Apple, $AAPL, iPhone sales in China fa...",3460,-0.600006
3,2024-01-11,ABBV,163.779999,163.779999,165.119995,163.085999,164.250000,4535649,0.113778,0.492681,AbbVie,0.4681,[{'d': 'Collaborations are critical to our bus...,50,-1.110001
4,2024-01-11,ABNB,139.449997,139.449997,141.199997,137.550003,140.710007,2351132,0.274910,0.272468,Airbnb,-0.0654,"[{'d': 'Airbnb’s “new improved, we really mean...",32,-0.309998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,2024-01-11,YUM,129.000000,129.000000,129.035004,127.599998,128.389999,1271579,0.315437,0.139413,Yum! Brands,0.4753,[{'d': 'Foreign Auto Barbell Approach: https:/...,9,-0.199997
499,2024-01-11,ZBH,122.379997,122.379997,124.809998,122.190002,123.230003,1643084,0.246834,0.290534,Zimmer Biomet,0.4019,[{'d': 'Zimmer Biomet adds Louis A. Shapiro ...,4,-0.850006
500,2024-01-11,ZBRA,255.029999,255.029999,256.869995,248.429993,254.009995,329485,0.315192,0.291633,Zebra Technologies,-0.1504,[{'d': '$ZBRA Office Depot Improves In-Store T...,0,-0.240005
501,2024-01-11,ZION,43.340000,43.340000,43.500000,42.299999,43.430000,1930588,0.442388,0.433599,Zions Bancorporation,0.0000,[],0,-0.590000


In [20]:
df_stock = pd.read_csv('SPY500_stock_2024_01_12.csv')

In [21]:
df_stock

,Unnamed: 0,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,ImpliedVolcalls,ImpliedVolputs,company,Sentiment_Vader,Posts_t,interaction
0,0,2024-01-12,A,130.600006,130.600006,131.610001,129.639999,130.309998,1088032,0.212166,0.303718,Agilent Technologies,0.0000,[{'d': 'Oakmark on Agilent Technologies $A:\n(...,2
1,1,2024-01-12,AAL,13.210000,13.210000,13.980000,13.190000,13.930000,80008335,0.250007,0.390631,American Airlines Group,0.7506,[{'d': '$AAL\nJefferies Upgrades American Airl...,24
2,2,2024-01-12,AAPL,185.919998,185.919998,186.740005,185.190002,186.059998,36923605,0.000010,0.209969,Apple Inc.,0.0878,[{'d': 'JUST IN: Microsoft $MSFT surpasses App...,19499
3,3,2024-01-12,ABBV,162.324997,162.324997,163.300003,162.195007,162.750000,3275493,0.354987,0.029307,AbbVie,0.5766,"[{'d': ""10 Dividend Growth Stocks\n\n💳 $V Visa...",108
4,4,2024-01-12,ABNB,137.139999,137.139999,140.990005,137.130005,139.350006,2324808,0.233406,0.000010,Airbnb,0.2960,"[{'d': 'BALOGH ARISTOTLE N (CTO) has sold 2,75...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,498,2024-01-12,YUM,129.119995,129.119995,129.419998,128.320007,129.479996,797625,0.284675,0.144296,Yum! Brands,-0.0037,[{'d': 'Analyst Upgrade:\n$AAL PT 18\n$CHWY PT...,65
499,499,2024-01-12,ZBH,123.070000,123.070000,124.750000,122.360001,123.690002,1268524,0.244392,0.329597,Zimmer Biomet,0.0000,[],0
500,500,2024-01-12,ZBRA,248.869995,248.869995,259.170013,243.559998,256.769989,619025,0.576786,0.319953,Zebra Technologies,0.4215,[{'d': 'Zebra Technologies to Unveil New Solut...,0
501,501,2024-01-12,ZION,42.110001,42.110001,43.660000,41.799999,43.580002,1682805,0.332038,0.000010,Zions Bancorporation,0.2960,"[{'d': 'Zions Bancorporation, National Associa...",0


In [22]:
from datetime import datetime as dt

check = []
changes = []
sumA = 0 
sumWrong = 0 
sumPlus = 0 
for i in range(len(df_stock['Symbol'])):  
    print(df_stock['Symbol'][i])
    ticker = df_stock['Symbol'][i]
    
    if type(df_stock['Date'][0]) == str:
        date_str = df_stock['Date'][0]
        date_format = '%Y-%m-%d'
        date_obj = dt.strptime(date_str, date_format)
        end = date_obj
    #     try: #  test if it is a weekend
    #         start = df_stock['Date'][0] - timedelta(days = 1) # weekday
    #         last_price = yf.download(ticker, start=start, end=end)['Adj Close'][0]
    #     except: 
        start = date_obj - timedelta(days = 1) # new year
    else:
        start = df_stock['Date'][0] - timedelta(days = 1) # new year
        end = df_stock['Date'][0]
    try: 
        last_price = yf.download(ticker, start=start, end=end)['Adj Close'][0]

#     last_price = yf.download(ticker, start=start, end=end)['Adj Close'][0]
        change = df_stock['Adj Close'][i] - last_price
        changes.append(float(change) )
    except: 
        print("can't get the last price!")
        changes.append(0)
#                 print(float(change))
    if float(change) > 0: sumPlus += 1
    if (df_stock['Sentiment_Vader'][i] >= 0.05 and float(change) > 0) or (df_stock['Sentiment_Vader'][i] <= - 0.05  and float(change) < 0): 
        sumA += 1
    if (df_stock['Sentiment_Vader'][i] >=  0.05  and float(change) < 0) or (df_stock['Sentiment_Vader'][i] <= - 0.05  and float(change) > 0): 
        sumWrong += 1
df_stock['Changes'] = changes
df_stock.to_csv(file_name, index = False)



A
[*********************100%***********************]  1 of 1 completed
AAL
[*********************100%***********************]  1 of 1 completed
AAPL
[*********************100%***********************]  1 of 1 completed
ABBV
[*********************100%***********************]  1 of 1 completed
ABNB
[*********************100%***********************]  1 of 1 completed
ABT
[*********************100%***********************]  1 of 1 completed
ACGL
[*********************100%***********************]  1 of 1 completed
ACN
[*********************100%***********************]  1 of 1 completed
ADBE
[*********************100%***********************]  1 of 1 completed
ADI
[*********************100%***********************]  1 of 1 completed
ADM
[*********************100%***********************]  1 of 1 completed
ADP
[*********************100%***********************]  1 of 1 completed
ADSK
[*********************100%***********************]  1 of 1 completed
AEE
[*********************100%*****************

[*********************100%***********************]  1 of 1 completed
CPB
[*********************100%***********************]  1 of 1 completed
CPRT
[*********************100%***********************]  1 of 1 completed
CPT
[*********************100%***********************]  1 of 1 completed
CRL
[*********************100%***********************]  1 of 1 completed
CRM
[*********************100%***********************]  1 of 1 completed
CSCO
[*********************100%***********************]  1 of 1 completed
CSGP
[*********************100%***********************]  1 of 1 completed
CSX
[*********************100%***********************]  1 of 1 completed
CTAS
[*********************100%***********************]  1 of 1 completed
CTLT
[*********************100%***********************]  1 of 1 completed
CTRA
[*********************100%***********************]  1 of 1 completed
CTSH
[*********************100%***********************]  1 of 1 completed
CTVA
[*********************100%*****************

[*********************100%***********************]  1 of 1 completed
HON
[*********************100%***********************]  1 of 1 completed
HPE
[*********************100%***********************]  1 of 1 completed
HPQ
[*********************100%***********************]  1 of 1 completed
HRL
[*********************100%***********************]  1 of 1 completed
HSIC
[*********************100%***********************]  1 of 1 completed
HST
[*********************100%***********************]  1 of 1 completed
HSY
[*********************100%***********************]  1 of 1 completed
HUBB
[*********************100%***********************]  1 of 1 completed
HUM
[*********************100%***********************]  1 of 1 completed
HWM
[*********************100%***********************]  1 of 1 completed
IBM
[*********************100%***********************]  1 of 1 completed
ICE
[*********************100%***********************]  1 of 1 completed
IDXX
[*********************100%**********************

[*********************100%***********************]  1 of 1 completed
NOC
[*********************100%***********************]  1 of 1 completed
NOW
[*********************100%***********************]  1 of 1 completed
NRG
[*********************100%***********************]  1 of 1 completed
NSC
[*********************100%***********************]  1 of 1 completed
NTAP
[*********************100%***********************]  1 of 1 completed
NTRS
[*********************100%***********************]  1 of 1 completed
NUE
[*********************100%***********************]  1 of 1 completed
NVDA
[*********************100%***********************]  1 of 1 completed
NVR
[*********************100%***********************]  1 of 1 completed
NWS
[*********************100%***********************]  1 of 1 completed
NWSA
[*********************100%***********************]  1 of 1 completed
NXPI
[*********************100%***********************]  1 of 1 completed
O
[*********************100%**********************

[*********************100%***********************]  1 of 1 completed
TSLA
[*********************100%***********************]  1 of 1 completed
TSN
[*********************100%***********************]  1 of 1 completed
TT
[*********************100%***********************]  1 of 1 completed
TTWO
[*********************100%***********************]  1 of 1 completed
TXN
[*********************100%***********************]  1 of 1 completed
TXT
[*********************100%***********************]  1 of 1 completed
TYL
[*********************100%***********************]  1 of 1 completed
UAL
[*********************100%***********************]  1 of 1 completed
UBER
[*********************100%***********************]  1 of 1 completed
UDR
[*********************100%***********************]  1 of 1 completed
UHS
[*********************100%***********************]  1 of 1 completed
ULTA
[*********************100%***********************]  1 of 1 completed
UNH
[*********************100%**********************

In [23]:
sumA, sumWrong, sumPlus

(158, 129, 260)

In [24]:
file_name

'SPY500_stock_2024_01_12.csv'

In [25]:
df_stock[df_stock['Sentiment_Vader']!=0]

,Unnamed: 0,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,ImpliedVolcalls,ImpliedVolputs,company,Sentiment_Vader,Posts_t,interaction,Changes
1,1,2024-01-12,AAL,13.210000,13.210000,13.980000,13.190000,13.930000,80008335,0.250007,0.390631,American Airlines Group,0.7506,[{'d': '$AAL\nJefferies Upgrades American Airl...,24,-1.380000
2,2,2024-01-12,AAPL,185.919998,185.919998,186.740005,185.190002,186.059998,36923605,0.000010,0.209969,Apple Inc.,0.0878,[{'d': 'JUST IN: Microsoft $MSFT surpasses App...,19499,0.330002
3,3,2024-01-12,ABBV,162.324997,162.324997,163.300003,162.195007,162.750000,3275493,0.354987,0.029307,AbbVie,0.5766,"[{'d': ""10 Dividend Growth Stocks\n\n💳 $V Visa...",108,-1.455002
4,4,2024-01-12,ABNB,137.139999,137.139999,140.990005,137.130005,139.350006,2324808,0.233406,0.000010,Airbnb,0.2960,"[{'d': 'BALOGH ARISTOTLE N (CTO) has sold 2,75...",0,-2.309998
5,5,2024-01-12,ABT,113.930000,113.930000,114.930000,113.459999,114.169998,2809487,0.000010,0.157235,Abbott,0.8442,[{'d': 'Abbott Laboratories Shares Near 52-Wee...,10,0.430000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,494,2024-01-12,XEL,61.310001,61.310001,61.619999,61.055000,61.349998,2499858,0.179696,1.127446,Xcel Energy,0.4707,[{'d': '📈 Top 5 Nasdaq 100 Gainers 🇺🇸\nMARVELL...,3,0.310001
495,495,2024-01-12,XOM,99.940002,99.940002,100.644997,99.170303,100.139999,13845830,0.000010,0.081064,ExxonMobil,-0.1846,[{'d': '2 energy biggies with solid free cashf...,1,1.270004
498,498,2024-01-12,YUM,129.119995,129.119995,129.419998,128.320007,129.479996,797625,0.284675,0.144296,Yum! Brands,-0.0037,[{'d': 'Analyst Upgrade:\n$AAL PT 18\n$CHWY PT...,65,0.119995
500,500,2024-01-12,ZBRA,248.869995,248.869995,259.170013,243.559998,256.769989,619025,0.576786,0.319953,Zebra Technologies,0.4215,[{'d': 'Zebra Technologies to Unveil New Solut...,0,-6.160004
